In [ ]:
// autoe

from keras.datasets import mnist
import numpy as np
(train_images,__),(test_images,_)=mnist.load_data()
print(train_images.shape)
print(test_images.shape)

import matplotlib.pyplot as plt
plt.imshow(test_images[0].reshape(28,28))
plt.gray()

train_images=train_images.astype('float32')/255.
test_images=test_images.astype('float32')/255.

train_images=train_images.reshape((len(train_images),np.prod(train_images.shape[1:])))
test_images=test_images.reshape((len(test_images),np.prod(test_images.shape[1:])))
print(train_images.shape)
print(train_images.shape)

from keras.layers import Input,Dense
from keras.models import Model
encoding_dim=32
input_layer=Input(shape=(784,))
encoder_layer1=Dense(encoding_dim,activation='relu')(input_layer)
decoder_layer1=Dense(784,activation='sigmoid')(encoder_layer1)
autoencoder=Model(input_layer,decoder_layer1)
autoencoder.summary()

encoder=Model(input_layer,encoder_layer1)

encoded_input=Input(shape=(encoding_dim,))
decoder_layer=autoencoder.layers[-1]
decoder=Model(encoded_input,decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam',loss='binary_crossentropy')

autoencoder.fit(train_images,train_images,
epochs=60,
batch_size=256,
shuffle=True,
validation_data=(test_images,test_images))

encoded_imgs=encoder.predict(test_images)
print(encoded_imgs.shape)

decoded_imgs=decoder.predict(encoded_imgs)
print(decoded_imgs.shape)

import matplotlib.pyplot as plt

n = 10  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(test_images[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()



In [ ]:
// backpr

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Sigmoid function
def nlinear(x,deriv=False):
  if(deriv==True):
    return x*(1-x)
  return 1/(1+np.exp(-x))

# Input
X=np.array([[1,1,0],[0,1,1],[0,0,1],[1,1,0]])
# Output
y=np.array([[1,0,0,1]]).T
# Seed for random number distribution
#np.random.seed(1)
# Weights initialization
synapse0=2*np.random.random((3,1))-1

for i in range(1000):
  # Forward propagation
  layer0=X
  layer1=nlinear(np.dot(layer0,synapse0))
  # Error
  layer1_error=y-layer1
  # Multiply with error backpropagated
  layer1_delta=layer1_error*nlinear(layer1,True)
  # Update wts
  synapse0+=np.dot(layer0.T,layer1_delta)

print("Predicted Output:")
print(layer1)
print("Actual Output:")
print(y)

df=[y,layer1]
df

plt.plot(y,layer1)



In [ ]:
// cnn

import tensorflow as tf
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

import matplotlib.pyplot as plt
print(y_train[1])
plt.imshow(x_train[1],cmap='Greys')

x_train.shape

x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_test=x_test.reshape(x_test.shape[0],28,28,1)
input_shape=(28,28,1)
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
x_train/=255
x_test/=255
print(x_train.shape[0])
print(x_test.shape[0])

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,kernel_size=(3,3),input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(256,activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(10,activation=tf.nn.softmax))

model.summary()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(x=x_train,y=y_train, epochs=10)

model.evaluate(x_test,y_test)

image_index=6630
plt.imshow(x_test[image_index].reshape(28,28),cmap='Greys')
pred=model.predict(x_test[image_index].reshape(1,28,28,1))
print(pred.argmax())


In [ ]:
// lstm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM, Dense,Dropout
from sklearn.preprocessing import LabelEncoder
import csv

imdb = pd.read_csv('/content/drive/MyDrive/Data/IMDB Dataset.csv',engine="python")
imdb.head()

imdb.sentiment.value_counts()

text=imdb['review'][10]
print(text)
print("---------------------")
print(word_tokenize(text))

corpus=[]
for text in imdb['review']:
  words=[word.lower() for word in word_tokenize(text)]
  corpus.append(words)

num_words=len(corpus)
print(num_words)

imdb.shape

train_size=int(imdb.shape[0]*0.8)
X_train=imdb.review[:train_size]
y_train=imdb.sentiment[:train_size]

X_test=imdb.review[train_size:]
y_test=imdb.sentiment[train_size:]

tokenizer=Tokenizer(num_words)
tokenizer.fit_on_texts(X_train)
X_train=tokenizer.texts_to_sequences(X_train)
X_train=pad_sequences(X_train,maxlen=128, truncating='post',padding='post')

X_train[0],len(X_train[0])

X_test=tokenizer.texts_to_sequences(X_test)
X_test=pad_sequences(X_test,maxlen=128,truncating='post',padding='post')

X_test[0],len(X_test[0])

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.transform(y_test)

model=Sequential()
model.add(Embedding(input_dim=num_words,output_dim=100,input_length=128,trainable=True))
model.add(LSTM(100,dropout=0.1,return_sequences=True))
model.add(LSTM(100,dropout=0.1))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

history=model.fit(X_train,y_train,epochs=5,batch_size=64,validation_data=(X_test,y_test))

plt.figure(figsize=(16,5))
epochs=range(1,len(history.history['accuracy'])+1)
plt.plot(epochs,history.history['loss'],'b',label='Training loss')
plt.plot(epochs,history.history['val_loss'],'b',label='Validation loss')
plt.legend()
plt.show()

#validation_sentence=['this movie was not good at all. it had some good parts like the acting was pretty good but story was not impressive at all']
validation_sentence=['i can watch the movie forever just because of beuty of cinematography']
validation_sentence_tokened=tokenizer.texts_to_sequences(validation_sentence)
val_sent_pad=pad_sequences(validation_sentence_tokened,maxlen=128,truncating='post',padding='post')
print(validation_sentence[0])
print(model.predict(val_sent_pad)[0])



In [ ]:
// mcculloch


def fire(theta, sum):
  if sum >= theta:
    return 1
  else:
    return 0

input = [[1,1,1],[0,0,0],[1,1,0],[0,0,1]]

for i in input:
  sum = 0
  for j in i:
    sum += j
  print("AND (",i,") = ", fire(len(i), sum))

def fire(theta, sum):
  if sum >= theta:
    return 1
  else:
    return 0

input = [[1,1,1],[0,0,0],[1,1,0],[0,0,1]]

for i in input:
  sum = 0
  for j in i:
    sum += j
  print("OR (",i,") = ", fire(1, sum))

def fire(theta, sum):
  if sum == theta:
    return 1
  else:
    return 0

input = [1,0]

for i in input:
  print("NOT (",i,") = ", fire(0, i))



In [ ]:
// perceptron


import numpy as np

inputs = np.array([[0,0],[1,0]])
weights = np.array([1,1])
bias = -0.5

def ORs(x, wt, b):
  fx = np.dot(wt, x) + b
  if fx >= 0:
    return 1
  else:
    return 0

for i in inputs:
  print("OR (", i, ") = ", ORs(i, weights, bias))



In [ ]:
// sgg

import pandas as pd
from IPython.display import display

red_wine=pd.read_csv('red-wine.csv')
df_train=red_wine.sample(frac=0.7,random_state=0)
df_valid=red_wine.drop(df_train.index)
display(df_train.head(4))

max_=df_train.max(axis=0)
min_=df_train.min(axis=0)
df_train=(df_train-min_)/(max_-min_)
df_valid=(df_valid-min_)/(max_-min_)

X_train = df_train.drop('quality', axis=1)
X_valid = df_valid.drop('quality', axis=1)

y_train = df_train['quality']
y_valid = df_valid['quality']

print(X_train.shape, X_valid.shape)

from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=[11]),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1),
])

model.compile(
    optimizer='sgd',
    loss='mse',
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=256,
    epochs=100,
    verbose = 1
)

history_df=pd.DataFrame(history.history)
history_df['loss'].plot()



